In [ ]:
import os  # for file paths
import pandas as pd
import awswrangler as wr
import pydbtools as pydb  # see https://github.com/moj-analytical-services/pydbtools

# few things for viewing dataframes better
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 900)
pd.set_option("display.max_colwidth", 200)

In [ ]:
# define some variables to be used throughout the notebook
db1 = "familyman_live_v4" #database where Familyman data is stored
db2 = "fcsq" #database where tables created as part of FCSQ processing are stored where required
#snapshot dates and publication period are set in the main run file. However, if running this notebook independently you will need to set them here
snapshot_date = "2024-04-02"
#pub_year = 2011
#pub_qtr = 4

### Children Act applicants

In [ ]:
#Get childre act applications data and link applicants to it
pydb.create_temp_table( 
f"""
SELECT 
  a.year,
  a.main_case_type,
  g.gender_code,
  count(*) AS count
FROM
  {db2}.ca_applications a
  LEFT JOIN {db2}.ca_applicants g
    ON a.case_number = g.case_number
WHERE 
  a.Year = 2023
  and a.quarter <> 4
GROUP BY
  a.year,
  a.main_case_type,
  g.gender_code

""",

"ca_apps_gender")

In [ ]:
#Pull out all gender types just to see that the total rougly tallies to FCSQ
pydb.read_sql_query ("select * from __temp__.ca_apps_gender")

### Domestic Violence applicants

In [ ]:
#Gender is saved in the DV apps gender table (thankfully!)
#Distinct event to get to application level rather than orders applied for
pydb.create_temp_table( 
f"""
SELECT 
DISTINCT
  year,
  event,
  gender
FROM
  {db2}.dv_app_gender
WHERE 
  Year = 2023
  and quarter <> 4
GROUP BY
  year,
  event,
  gender

""",

"dv_apps_gender")

In [ ]:
pydb.create_temp_table( 
f"""
SELECT 
  year,
  gender,
  count (*) AS count
FROM
  __temp__.dv_apps_gender
GROUP BY
  year,
  gender

""",

"dv_apps_gender_agg")

In [ ]:
pydb.read_sql_query ("select * from __temp__.dv_apps_gender_agg")

### Adoption applicants

In [ ]:
#Get adoption applications data and link applicants to it
pydb.create_temp_table( 
f"""
SELECT 
  a.year,
  g.gender,
  count(*) AS count
FROM
  {db2}.adopt_application_5 a
  LEFT JOIN {db2}.adoptions_parties g
    ON a.case_number = g.case_number
WHERE 
  a.Year = 2023
  and a.quarter <> 4
  AND g.role_model = 'APLZ'
GROUP BY
  a.year,
  g.gender

""",

"adoption_apps_gender")

In [ ]:
pydb.read_sql_query ("select * from __temp__.adoption_apps_gender")

### Children Act timeliness - timeliness code will need to be re-run to create the relevant temp tables

In [ ]:
#Group by number of weeks and add a count of how many for each week so we can eventually take the highest
pydb.create_temp_table( 
f"""
SELECT 
  weeks,
  count (*) AS count
FROM
  __temp__.ca_timeliness_all
WHERE 
  year = 2023 
  AND order_case_type = 'P'
  AND quarter <> 4
GROUP BY
  weeks

""",
"priv_weeks_count")

In [ ]:
#get the number of weeks with the highest count
pydb.read_sql_query ("select MAX (count) AS mode FROM __temp__.priv_weeks_count")

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
# Getting the complete distribution of private law weeks
#get the number of weeks with the highest count
pydb.read_sql_query ("select * FROM __temp__.priv_weeks_count order by weeks")

In [ ]:
#mannually put in number above - most common is 0 weeks
pydb.read_sql_query ("select weeks from __temp__.priv_weeks_count where count = 6170")

### DV timeliness

In [ ]:
#Group by number of weeks and add a count of how many for each week so we can eventually take the highest
pydb.create_temp_table( 
f"""
SELECT 
  ROUND(wait_weeks) as weeks,
  count (*) AS count
FROM
  fcsq.dv_apps_and_orders_match
WHERE 
  year = 2023
  and month not in (10, 11, 12)
GROUP BY
  ROUND(wait_weeks)

""",
"dv_weeks_count")

In [ ]:
#get the number of weeks with the highest count
pydb.read_sql_query ("select MAX (count) AS mode FROM __temp__.dv_weeks_count")

In [ ]:
#get the complete distribution of domestic violence weeks
pydb.read_sql_query ("select * FROM __temp__.dv_weeks_count order by weeks")

In [ ]:
#mannually put in number above - most common is 0 weeks
pydb.read_sql_query ("select weeks from __temp__.dv_weeks_count where count = 14088")

### Adoption timeliness 

In [ ]:
#Group by number of weeks and add a count of how many for each week so we can eventually take the highest
pydb.create_temp_table( 
f"""
SELECT 
  ROUND(wait_weeks) as weeks,
  count (*) AS count
FROM
  fcsq.ADOPT_APPS_AND_ORDERS_MATCH
WHERE 
  year = 2023 
  and quarter <> 4
GROUP BY
  ROUND(wait_weeks)

""",
"adopt_weeks_count")

In [ ]:
#get the number of weeks with the highest count
pydb.read_sql_query ("select MAX (count) AS mode FROM __temp__.adopt_weeks_count")

In [ ]:
# get full distribution
pydb.read_sql_query ("select * FROM __temp__.adopt_weeks_count order by weeks")

In [ ]:
#mannually put in number above - most common is 0 weeks
pydb.read_sql_query ("select weeks from __temp__.adopt_weeks_count where count = 171")